# Лабораторная работа 7. Инструментарий Stable Baselines 3. Алгоритмы, функции обратного вызова, запись видео, подбор значений гиперпараметров


Ну что, ребята, вот и пришло наше время использовать "фит + предикт"!!!

В завершающей лабораторной работе мы познакомимся с классическим инструментом для решения задач обучения с подкреплением.

Мы решим 3 задачи в 3 окружениях. Эти окружения могут быть посложнее тех, которые мы использовали ранее. Первое решим с помощью DQN. Второе с помощью PPO - усовершенствованной версии алгоритма REINFORCE. И еще одно помощью DDPG, который рассматривался в предыдущей ЛР.

Крутость библиотеки Stable Baselines не только в эффективных реализациях различных продвинутых методов обучения с подкреплением, но и в том, что она предлагает дополнительные возможности для упрощения процесса обучения. Мы с вами познакомимся с функциями обратного вызова и запишем видео с агентом в окружении.

Также, в этой лабораторной работе именно вы определяете реальность и все реализуете самостоятельно. Связано это с тем, что во многих местах вы будете заниматься копипастом. Вы можете создавать свои ячейки и писать код так, как хотите. Наша цель понять, как библиотека Stable Baselines позволяет нам быстро и удобно получать готовые модели для решения различных задач обучения с подкреплением. Поехали!

## Установка библиотеки

In [1]:
# %pip install stable-baselines3[extra]
# %pip install box2d box2d-kengz
# %pip install box2d-py
# %pip install gym pyvirtualdisplay
# %pip install pygame
# %pip install pyglet

В этой работе вы будете часто обращаться к документации: https://stable-baselines3.readthedocs.io/en/master/index.html

## DQN и запись видео в Stable Baselines

Создайте любое окружение, которое может быть решено с помощью DQN (в помощь вам и информация из документации к Stable Baselines).

Выполните обучение модели в окружении. Установите желаемое количество шагов для обучения. Сохраните модель после обучения на ваш Google Drive или скачайте ее на компьютер с помощью Colab (подсказка: from google.colab import files и т.д.)

С помощью метода для оценки стратегии, оцените качество модели. Выполнить это нужно с помощью Stable Baselines. Сыграйте пару сотен игр и получите среднее значение и среднеквадратичное отклонение

Запишите видео с работой обученной модели. Сохраните его на Google Drive или загрузите на компьютер. При проверке ЛР видео просматриваются.

In [2]:
# ваш код
# нет, не мой))) https://stable-baselines3.readthedocs.io/en/master/guide/examples.html
import gym

from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from stable_baselines3.common.callbacks import EvalCallback

# Create environment
env = DummyVecEnv([lambda: gym.make("LunarLander-v2")])

# env = VecVideoRecorder(env, 'video',
#                        record_video_trigger=lambda x: x == 0, video_length=100,
#                        name_prefix=f"firstModel")
env.reset()

# Instantiate the agent
model = DQN("MlpPolicy", env)
# Train the agent and display a progress bar
# model.learn(total_timesteps=int(1e5))
model.learn(total_timesteps=int(1e5))
# Save the agent
model.save("model/dqn_lunar")
del model  # delete trained model to demonstrate loading

# Load the trained agent
# NOTE: if you have loading issue, you can pass `print_system_info=True`
# to compare the system on which the model was trained vs the current one
# model = DQN.load("dqn_lunar", env=env, print_system_info=True)
model = DQN.load("model/dqn_lunar", env=env)

# Evaluate the agent
# NOTE: If you use wrappers with your environment that modify rewards,
#       this will be reflected here. To evaluate with original rewards,
#       wrap environment in a "Monitor" wrapper before other wrappers.
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

# # Enjoy trained agent
# vec_env = model.get_env()
# obs = vec_env.reset()
# for i in range(1000):
#     action, _states = model.predict(obs, deterministic=True)
#     obs, rewards, dones, info = vec_env.step(action)
#     vec_env.render()

print('mean_reward', mean_reward, sep=' : ')
print('std_reward', std_reward, sep=' : ')

/home/sega/.local/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward : -127.44996056969248
std_reward : 52.22871020521623


In [22]:
from stable_baselines3.common.vec_env import VecVideoRecorder

video_length=1000

env = VecVideoRecorder(env, 'video',
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix="firstModel")

state = env.reset()

for _ in range(video_length + 1):
  action = model.predict(state)[0]
  state, _, _, _ = env.step(action)
# Save the video
env.close()

AttributeError: 'PendulumEnv' object has no attribute 'num_envs'

## PPO и функции обратного вызова в Stable Baselines

По аналогии с DQN - самостоятельно выберите любое окружение и обучите агента с помощью алгоритма PPO. В этот раз надо будет выполнить обучение 3 раза.

В прошлый раз мы сохраняли модель после завершения обучения. В этот раз не делайте так!

В библиотеке Stable Baselines есть такая вещь как функции обратного вызова. Вы можете реализовывать свои, но уже есть несколько предопределенных. Найдите калбек, который будет сохранять вашу модель через каждые n шагов. Обучите модель с использованием такого калбека.

In [4]:
# ваш код с обучением, последовательным сохранением промежуточных моделей и тестированием конечной модели
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import CheckpointCallback

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path="./logs/",
  name_prefix="rl_model",
  save_replay_buffer=True,
  save_vecnormalize=True, # нужно для тех env которые используют VecNormalize
)

# Parallel environments
env = DummyVecEnv([lambda: gym.make("CartPole-v1")])

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25000, callback=checkpoint_callback)

Using cpu device
-----------------------------
| time/              |      |
|    fps             | 803  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 638         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008913951 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00773     |
|    learning_rate        | 0.0003      |
|    loss                 | 5.36        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0159     |
|    value_loss           | 53.4        |
-----------------------------------------
-----------------

Теперь найдите калбек, который позволяет оценивать модель каждые n шагов и в итоге сохранить лучшую модель. Не забудьте указать папки для сохранения лучшей модели и для сохранения результатов тестирования

In [5]:
# ваш код с обучением, последовательным оцениванием промежуточных моделей и тестированием конечной модели
from stable_baselines3.common.callbacks import EvalCallback

n = 500

# чего он выводит: номер шага, среднюю награду, среднеквадратичное отклонение награды,
# среднуюю длительность игры, среднеквадратичное отклонение длительности
# ну понятно что награда будет совпадать с длительностью
# потому что енв=cartpole и там за каждый шаг длительность +1 и награда +1
# https://github.com/hill-a/stable-baselines/blob/master/stable_baselines/common/callbacks.py
eval_callback = EvalCallback(env, best_model_save_path="./logs/best-model",
                             log_path="./logs/", eval_freq=n,
                             deterministic=True, render=False)

model = PPO("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=25000, callback=eval_callback)

/home/sega/.local/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Eval num_timesteps=500, episode_reward=8.60 +/- 0.80
Episode length: 8.60 +/- 0.80
New best mean reward!
Eval num_timesteps=1000, episode_reward=8.40 +/- 0.49
Episode length: 8.40 +/- 0.49
Eval num_timesteps=1500, episode_reward=8.40 +/- 0.49
Episode length: 8.40 +/- 0.49
Eval num_timesteps=2000, episode_reward=8.80 +/- 0.75
Episode length: 8.80 +/- 0.75
New best mean reward!
Eval num_timesteps=2500, episode_reward=153.00 +/- 27.16
Episode length: 153.00 +/- 27.16
New best mean reward!
Eval num_timesteps=3000, episode_reward=174.40 +/- 48.78
Episode length: 174.40 +/- 48.78
New best mean reward!
Eval num_timesteps=3500, episode_reward=144.20 +/- 12.73
Episode length: 144.20 +/- 12.73
Eval num_timesteps=4000, episode_reward=143.40 +/- 18.10
Episode length: 143.40 +/- 18.10
Eval num_timesteps=4500, episode_reward=305.60 +/- 143.13
Episode length: 305.60 +/- 143.13
New best mean reward!
Eval num_timesteps=5000, episode_reward=292.20 +/- 146.30
Episode length: 292.20 +/- 146.30
Eval num_ti

А теперь вы вправе использовать любые калбеки, которые хотите, но не менее двух. Stable Baselines позволяет обучать модель с несколькими калбеками. Изучите, как это происходит, и обучите еще одну модель с несколькими калбеками.

In [6]:
# ваш код с обучением, с несколькими калбеками на ваш выбор
# КАЛбеки вызываются последовательно в том порядке, в котором были переданы
# при прокидывании листа в параметр callback из него (листа в смысле) автоматически
# формируется CallbackList
# https://stable-baselines3.readthedocs.io/en/master/guide/callbacks.html
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.callbacks import EveryNTimesteps

class MyCoolCallback(BaseCallback):
  def __init__(self, eval_env):
    self.n_calls = 0
    self.eval_env = eval_env
    self.max_reward = -float('inf')
  
  def _on_step(self) -> bool:
    """
    This method will be called by the model after each call to `env.step()`.
    For child callback (of an `EventCallback`), this will be called
    when the event is triggered.
    :return: (bool) If the callback returns False, training is aborted early.
    """
    reward, _ = evaluate_policy(self.model, self.eval_env,
                                 n_eval_episodes=1,
                                 render=False,
                                 deterministic=True,
                                 return_episode_rewards=True)
    self.n_calls = self.n_calls + 1

    print('---')
    print('step :>> ', self.model.num_timesteps, '|', 'reward :>>', reward[0], end=' ')
    if reward[0] >= self.max_reward:
      self.max_reward = reward[0]
      print('(new max!)')
    else:
      print()

    return True

every_n_timestep = EveryNTimesteps(500, MyCoolCallback(env))

model = PPO("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=25000, callback=[checkpoint_callback, every_n_timestep])

---
step :>>  500 | reward :>> 28.0 (new max!)
---
step :>>  1000 | reward :>> 23.0 
---
step :>>  1500 | reward :>> 25.0 
---
step :>>  2000 | reward :>> 28.0 (new max!)
---
step :>>  2500 | reward :>> 500.0 (new max!)
---
step :>>  3000 | reward :>> 363.0 
---
step :>>  3500 | reward :>> 311.0 
---
step :>>  4000 | reward :>> 414.0 
---
step :>>  4500 | reward :>> 125.0 
---
step :>>  5000 | reward :>> 75.0 
---
step :>>  5500 | reward :>> 114.0 
---
step :>>  6000 | reward :>> 75.0 
---
step :>>  6500 | reward :>> 111.0 
---
step :>>  7000 | reward :>> 139.0 
---
step :>>  7500 | reward :>> 386.0 
---
step :>>  8000 | reward :>> 388.0 
---
step :>>  8500 | reward :>> 218.0 
---
step :>>  9000 | reward :>> 264.0 
---
step :>>  9500 | reward :>> 243.0 
---
step :>>  10000 | reward :>> 483.0 
---
step :>>  10500 | reward :>> 500.0 (new max!)
---
step :>>  11000 | reward :>> 152.0 
---
step :>>  11500 | reward :>> 269.0 
---
step :>>  12000 | reward :>> 500.0 (new max!)
---
step :>>  12

## DDPG и подбор гиперпараметров в Stable Baselines

Выберите подходящее окружение. Обучите модель с использованием алгоритма DDPG. Оцените модель, сыграв пару сотен игр.

Сохраните полученную модель на Google Drive или загрузите ее на свой компьютер.

In [8]:
# ваш код с обучением и тестированием модели, а также сохранением ее в Google Drive или на компьютере
import numpy as np

from stable_baselines3 import DDPG

env = gym.make("Pendulum-v1")

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]

model = DDPG("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000, log_interval=10)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.33e+03 |
| time/              |           |
|    episodes        | 10        |
|    fps             | 151       |
|    time_elapsed    | 13        |
|    total_timesteps | 2000      |
| train/             |           |
|    actor_loss      | 53.4      |
|    critic_loss     | 0.073     |
|    learning_rate   | 0.001     |
|    n_updates       | 1800      |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.16e+03 |
| time/              |           |
|    episodes        | 20        |
|    fps             | 143       |
|    time_elapsed    | 27        |
|    total_timesteps | 4000      |
| train/             |           |
|    actor_loss      | 86.5    

Обратитесь к статье из документации, в которой описывается тюнинг модели: https://stable-baselines.readthedocs.io/en/master/guide/rl_zoo.html

Используйте эту информацию, чтобы обучить оптимальную DDPG-модель для вашего окружения. Используйте описанный в статье репозиторий. Поработайте с файлом train.py, чтобы подобрать оптимальные значения гиперпараметров. Используйте файл enjoy.py, чтобы протестировать вашу модель. Подсказка: обученные модели сохраняются в папку /logs.

In [14]:
# ваш код
!apt-get install swig cmake libopenmpi-dev zlib1g-dev ffmpeg
%pip install stable-baselines box2d box2d-kengz pyyaml pybullet optuna pytablewriter
!git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Defaulting to user installation because normal site-packages is not writeable
  Using cached Box2D-kengz-2.3.3.tar.gz (425 kB)
  Preparing metadata (setup.py) ... done
  Using cached optuna-3.0.4-py3-none-any.whl (348 kB)
  Using cached pytablewriter-0.64.2-py3-none-any.whl (106 kB)
  Using cached colorlog-6.7.0-py2.py3-none-any.whl (11 kB)
  Using cached cliff-3.10.1-py3-none-any.whl (81 kB)
  Using cached SQLAlchemy-1.4.44-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached alembic-1.8.1-py3-none-any.whl (209 kB)
  Using cached cmaes-0.9.0-py3-none-any.whl (23 kB)
  Using cached tabledata-1.3.0-py3-none-any.whl (11 kB)
  Using cached mbstrdecoder-1.1.1-py3-none-any.whl (7.7 kB)
  Using cached pathvalidate-2.5.2-py3-none-any.whl (20 kB)
  Using

In [10]:
%pip install tensorflow==1.13.2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
!python3.7 rl-baselines3-zoo/train.py --algo ddpg --env Pendulum-v1

========== Pendulum-v1 ==========
Seed: 1206920771
Loading hyperparameters from: /home/sega/study/reinforcement-learning/lab_7/rl-baselines3-zoo/hyperparams/ddpg.yml
Default hyperparameters for environment (ones being tuned will be overridden):
OrderedDict([('buffer_size', 200000),
             ('gamma', 0.98),
             ('gradient_steps', -1),
             ('learning_rate', 0.001),
             ('learning_starts', 10000),
             ('n_timesteps', 20000),
             ('noise_std', 0.1),
             ('noise_type', 'normal'),
             ('policy', 'MlpPolicy'),
             ('policy_kwargs', 'dict(net_arch=[400, 300])'),
             ('train_freq', [1, 'episode'])])
Using 1 environments
Creating test environment
Applying normal noise with std 0.1
Using cpu device
Log path: logs/ddpg/Pendulum-v1_2
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.25e+03 |
| time/              |           |
|    epi

In [1]:
from stable_baselines3 import DDPG
import gym

env = gym.make('Pendulum-v1')
model = DDPG.load("logs/ddpg/Pendulum-v1_2/Pendulum-v1.zip", env=env)
state = env.reset()

while True:
  action = model.predict(state)[0]
  state, _, done, _ = env.step(action)
  if done: break;

env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Теперь вы знаете, как с помощью одной библиотеки и одного репозитория решать самые разные задачи. Напоминаю, что никто не запрещает вам создавать собственные окружения и решать задачи в них!

## Последнее слово Гуненкова

<b>Дорогой студент, дорогой друг (подруга)!</b>
<br/><br/>
Хочу поздравить тебя с выполнением всех заданий этого экспериментального курса. Благодарю тебя за то, что ты не сдался (не сдалась) и дошел до конца. 
<br/><br/>
Я надеюсь, что ты разобрался в основах обучения с подкреплением и теперь имеешь практические навыки в создании собственных крутых моделей. 
<br/><br/>
Ты принадлежишь к числу тех, кто впервые прошел этот курс. Возможно, эти лабораторные больше никто не будет выполнять. А может быть, кафедра действительно будет их использовать для обучения на 4 курсе.

Эти лабораторные не сделаны по книгам или по урокам. Они сделаны человеком, который вместе с тобой погрузился в эту новую дисциплину.
<br/><br/>
<b>Если тебе есть что сказать именно по поводу этих заданий</b> (скорее всего тебя попросят пройти традиционные опросы по каждому преподавателю, но здесь это только между нами), прошу писать сюда: https://forms.gle/3nqvHbtAMo8qghmXA
<br/><br/>
Я проанализирую твое мнение и обязательно учту его в дальнейшей работе. Спасибо!
<br/><br/>
А теперь, нам пора прощаться.
<br/><br/>
<b>Желаю успешной подготовки и защиты ВКР и поступления в магистратуру</b> 
<br/><br/>
До свидания! :)

## Задания лабораторной работы

### Основные задания

1. Выполните все промежуточные задания. <b> 6 баллов </b>
<br /><br />
Итого за лабораторную работу: 6 баллов.

### Замечания

1. Выполнять задания можно в Google Colab или локально в среде Jupiter Notebook. При выполнении задания локально не требуется настройка и установка зависимостей для виртуального дисплея (и использование кастомной функции visualize). При вызове метода env.render создастся окно с визуализацией.